### Paper

*[German also Hallucinates! Inconsistency Detection in News
Summaries with the Absinth Dataset](https://aclanthology.org/2024.lrec-main.680.pdf)*

In [1]:
from transformers import pipeline
from datasets import load_dataset
from tqdm.auto import tqdm
import random
import pandas as pd

### Model

*[Shahm/bart-german](https://huggingface.co/Shahm/bart-german)*

In [2]:
MAX_LEN = 2048

In [3]:
#summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
#summarizer = pipeline("summarization", model="facebook/mbart-large-cc25")
summarizer = pipeline("summarization", model="Shahm/bart-german")

In [4]:
def summarize_text(input_text):
    summary_text = summarizer(input_text[:MAX_LEN], min_length=25, max_length=200)[0]["summary_text"]
    return summary_text

### Dataset

*[jonathandechert/20Minuten](https://huggingface.co/datasets/jonathandechert/20Minuten)*

In [5]:
twenty_minutes_ds = load_dataset("jonathandechert/20Minuten", split="test")
twenty_minutes_ds

Dataset({
    features: ['original', 'simplification'],
    num_rows: 200
})

In [6]:
def sample_20minutes(n_samples=5):
    dataset_len = len(twenty_minutes_ds)
    sample_indices = random.sample(range(dataset_len), n_samples)
    for i in sample_indices:
        cur_text = twenty_minutes_ds[i]["original"]
        print(cur_text)
        print()

        summary_text = summarize_text(cur_text)
        print(summary_text)
        print("-"*100)


In [7]:
sample_20minutes(3)

Ich konnte meine Antidepressiva absetzenSandra N.* (33) Ich hatte mein zweites Burnout im Sommer 2019 und musste Antidepressiva und Entspannungstabletten nehmen. Ich bin per Zufall auf einen Post eines befreundeten Ernährungsberaters gestossen, wo dieser erklärt, wie eine gestörte Hormonsituation ein Burnout auslösen kann. Seit Oktober 2019 arbeite ich mit ihm zusammen und nehme Nahrungsergänzungsmittel. Einige Wochen nach Beginn der Zusammenarbeit konnte ich sowohl die Antidepressiva, als auch die Entspannungstabletten absetzen und habe nie wieder etwas davon gebraucht. Zudem führe ich eine App, wo ich mein Essen eingebe, um meine Werte zu erreichen. Mir ging es davor sehr schlecht. Ich bin während der Arbeit wegen meinen Schlafstörungen im Stehen fast eingeschlafen und konnte meine Augen kaum aufhalten. Nach drei Wochen ging es mir schon viel besser. Im April 2020 habe ich meine Burnout-Therapie beendet. Ich bin überglücklich und fühle mich so gut wie noch nie. Ich bin froh, dass ich

### Save Data

In [8]:
summary_df = pd.DataFrame(columns=["id","original","simplification","summary"])

for i in tqdm(range(len(twenty_minutes_ds))):
    cur_text = twenty_minutes_ds[i]["original"]
    cur_simpli = twenty_minutes_ds[i]["simplification"]
    cur_summary = summarize_text(cur_text)

    summary_df.loc[i] = [i, cur_text, cur_simpli, cur_summary]

  0%|          | 0/200 [00:00<?, ?it/s]

Your max_length is set to 200, but your input_length is only 26. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)
Your max_length is set to 200, but your input_length is only 11. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=5)


In [9]:
summary_df

,id,original,simplification,summary
0,0,Ich date jetzt schon seit längerer Zeit Timon....,Lars datet seit einiger Zeit Timon. Eigentlich...,"Timon entspricht eigentlich genau dem, was ich..."
1,1,Ich kann mich noch sehr gut an den Moment erin...,"Weil ihr Vater unfruchtbar ist, wurde G.C. 199...",Ich kann mich noch sehr gut an den Moment erin...
2,2,In tausenden Schweizer Privathaushalten kümmer...,"Putzhilfen, Babysitter und sonstige Haushaltsa...",Durchschnittlich bekommen sie für die Arbeit r...
3,3,Im Studio Gewichte stemmen und auf dem Laufban...,Clever-Fit schreibt die Shutdown-Zeit nicht au...,Viele Firmen setzen im Shutdown auf Abo-Verlän...
4,4,Gerade erst wurde Kanye West (43) noch inklusi...,Gerüchten zufolge sollen Kanye West (43) und I...,Nach seiner Scheidung von Reality-Star Kim Kar...
...,...,...,...,...
195,195,"Mit 32 Jahren, nach 18 Jahren als Profi, nach ...",Die Schweizer Tennisspielerin Timea Bacsinszky...,Die Lausannerin Timea Bacsinszky gewinnt die W...
196,196,Laut der grossen Sex-Umfrage von 20 Minuten (s...,Laut der Sex-Umfrage von 20 Minuten hatte fast...,Eine Umfrage von 20 Minuten hatte fast jede*r ...
197,197,Der niederländische Schiedsrichter Björn Kuipe...,Der älteste Schiedsrichter des Turniers pfeift...,Björn Kuipers leitet das Finale der EM an dies...
198,198,Ich habe die Frau online kennengelernt und wir...,Nando (26) lernte sein Date online kennen. Die...,Ich habe die Frau online kennengelernt und wir...


In [10]:
summary_df.to_csv("../data/summary.csv", index=False)